# Initialization


In [25]:
"""!pip install tensorflow==2.13.0
!pip install keras==2.13.1
!pip install tensorflow-addons==0.16.1"""

'!pip install tensorflow==2.13.0\n!pip install keras==2.13.1\n!pip install tensorflow-addons==0.16.1'

In [26]:
"""!pip install googletrans
!pip install langdetect
!pip install emojis
!pip install emoji
!pip install pyarabic
!pip install nltk
!pip install keras_tuner
!pip install wordcloud
!pip install prettytable
!pip install arabic_reshaper
!pip install python-bidi
!pip install openpyxl"""

'!pip install googletrans\n!pip install langdetect\n!pip install emojis\n!pip install emoji\n!pip install pyarabic\n!pip install nltk\n!pip install keras_tuner\n!pip install wordcloud\n!pip install prettytable\n!pip install arabic_reshaper\n!pip install python-bidi\n!pip install openpyxl'

In [27]:
import numpy as np
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas
import seaborn as sns
from sklearn import preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import re
import string
import matplotlib.cm as cm
from matplotlib import rcParams
from prettytable import PrettyTable
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import seaborn as sns
import os
from IPython.display import display, HTML
from googletrans import Translator
from langdetect import detect
import warnings
warnings.filterwarnings("ignore")
from nltk.corpus import stopwords
import re
import csv
import emojis
import pyarabic.araby as araby
from nltk.stem.isri import ISRIStemmer
from nltk.stem import SnowballStemmer
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.regularizers import l1_l2
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras import layers
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\me513\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Load Data

In [28]:
column_names = ["review_description", "rating"]
train_data = pd.read_excel(r"C:\Users\me513\Downloads\Depi-20241020T181025Z-001\Depi\train.xlsx", names=column_names)
test_data = pd.read_csv(r"C:\Users\me513\Downloads\Depi-20241020T181025Z-001\Depi\test _no_label.csv",encoding="utf-8",dtype={'review_description': str})

len(train_data), len(test_data)

(32036, 1000)

In [29]:
train_data[train_data.isnull().any(axis=1)].head()
test_data[train_data.isnull().any(axis=1)].head()
print(np.sum(train_data.isnull().any(axis=1)))
print(train_data.isnull().any(axis=0))

print(np.sum(test_data.isnull().any(axis=1)))
print(test_data.isnull().any(axis=0))

0
review_description    False
rating                False
dtype: bool
0
ID                    False
review_description    False
dtype: bool


In [30]:
train_data["review_description"] = train_data["review_description"].astype(str)
test_data["review_description"] = test_data["review_description"].astype(str)

train_data["review_description"] = train_data["review_description"].fillna({"review_description":"مجهول"})
test_data["review_description"]  =test_data["review_description"].fillna({"review_description":"مجهول"})

train_data, test_data

(                                      review_description  rating
 0      شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...      -1
 1      خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...       1
 2      تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...      -1
 3      فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...       1
 4      سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...      -1
 ...                                                  ...     ...
 32031  التطبيق اصبح سيء للغايه نقوم بطلب لا يتم وصول ...      -1
 32032                                         y love you       1
 32033      الباقه بتخلص وبشحن مرتين باقه اضافيه ١٠٠ جنيه      -1
 32034  تطبيق فاشل وصلني الطلب ناقص ومش ينفع اعمل حاجة...      -1
 32035                                    ليش ما يفتح معي       1
 
 [32036 rows x 2 columns],
        ID                                 review_description
 0       1  اهنئكم على خدمه العملاء في المحادثه المباشره م...
 1       2  ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسا

# preprocessing

In [31]:
for letter in '#.][!XR':
    train_data["review_description"] = train_data["review_description"].astype(str).str.replace(letter,'')
    test_data["review_description"] = test_data["review_description"].astype(str).str.replace(letter,'')

train_data.replace('', np.nan, inplace=True)

train_data["review_description"] = train_data["review_description"].fillna({"review_description":"مجهول"})
print(train_data["review_description"].isnull().sum())

3


In [32]:
def translate_EtoA(english_text):

    translator = Translator(service_urls=['translate.google.com'],to_lang="ar")
    translation = translator.translate(english_text)
    arabic_text = translation

    return arabic_text

In [33]:
def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [34]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [35]:
def remove_stop_words(text):
    stop_words = set(stopwords.words('arabic')) - {'نعم','لا', 'ليس', 'ليست', 'مش', 'ما','غير','أقبل','ليس','ليسا','ليست','ليستا','ليسوا','لست','لستم','لستما','لستن','لسن','لسنا','واو'}
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    return " ".join([word for word in tokens if word not in stop_words])

In [36]:
def build_emoji_dictionary(csv_file):
    emoji_dict = {}
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            emoji = row[0]
            text = row[1]
            emoji_dict[emoji] = text
    return emoji_dict

In [47]:
import emoji

def replace_emojis(emoji_dict, sentence):
    # Convert emojis in the sentence to their textual descriptions
    sentence = emoji.demojize(sentence)
    
    # Iterate through the emoji dictionary to replace with the custom format
    for key, value in emoji_dict.items():
        sentence = sentence.replace(':' + key + ':', '*' + value + '* ')
    
    return sentence


In [38]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [39]:
def decode_emojis(text):
    decoded_text = emojis.decode(text)
    return decoded_text

In [40]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [41]:
def remove_diactrics(text):
    return araby.strip_diacritics(text)

In [42]:
def processPost(text,emoji_dict):
    text = re.sub(r"\s+", " ", str(text).strip())
    text = re.sub('@[^\s]+', ' ', text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    text= remove_punctuations(text)
    text = normalize_arabic(text)
    text = replace_emojis(emoji_dict, text)
    text= remove_punctuations(text)
    text = remove_repeating_char(text)
    text = remove_stop_words(text)
    text = remove_diactrics(text)
    return text

In [48]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

import emoji

emoji_dict = build_emoji_dictionary(r"C:\Users\me513\Downloads\Depi-20241020T181025Z-001\Depi\emojis.csv")

train_data["review_description"] = train_data["review_description"].fillna({"review_description":"مجهول"})
print(train_data["review_description"].isnull().sum())
train_data["review_description"] = train_data["review_description"].apply(lambda x: processPost(x,emoji_dict))
test_data["review_description"] = test_data["review_description"].apply(lambda x: processPost(x,emoji_dict))

3


In [49]:
train_data.replace('', np.nan, inplace=True)
train_data["review_description"] = train_data["review_description"].fillna({"review_description":"مجهول"})
print("train data",train_data["review_description"].isnull().sum())

test_data.replace('', np.nan, inplace=True)
test_data["review_description"] = test_data["review_description"].fillna({"review_description":"مجهول"})
print("test data", test_data["review_description"].isnull().sum())

train data 247
test data 6


In [50]:
tokenizer = RegexpTokenizer(r'\w+')
train_data["review_description"] = train_data["review_description"].apply(str)
train_data["review_description"] = train_data["review_description"].apply(tokenizer.tokenize)

In [51]:
def stem(text):
    stemmed = []
    for word in text:
        stemmed.append(stemmer.stem(word))
    return stemmed

In [52]:
def engstem(text):
    e_stemmer = SnowballStemmer("english")
    stemmed = []
    for word in text:
        stemmed.append(e_stemmer.stem(word))
    return stemmed

In [53]:
stemmer = ISRIStemmer()

train_data["review_description"] = train_data["review_description"].apply(engstem)
train_data["review_description"] = train_data["review_description"].apply(lambda x: ' '.join(x))

train_data["review_description"]

0        شركه زباله سواقين بتبرشم مفيش حتي رقم لشكاوي ا...
1        خدمه الدفع طريق الكي نت توقفت عندي اصبح فقط ال...
2        تطبيق غبي جاري حذفه عاملين اكواد خصم نستخدمها ...
3        فعلا تطبيق متاز امكانيه يتيح لمستخدم التطبيق ا...
4              سيء جدا اسعار رسوم التوصيل لا تمت لواقع صله
                               ...                        
32031    التطبيق اصبح سيء لغايه نقوم بطلب لا يتم وصول ا...
32032                                           y love you
32033                 الباقه بتخلص وبشحن مرتين باقه اضافيه
32034    تطبيق فاشل وصلني الطلب ناقص ومش ينفع اعمل حاجه...
32035                                      ليش ما يفتح معي
Name: review_description, Length: 32036, dtype: object

# Tokenization and Embedding

In [54]:
X_train = train_data['review_description']
y_train = train_data['rating']
x_test = test_data['review_description'].astype(str).tolist()

In [55]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

X_tokenized = tokenizer.texts_to_sequences(X_train)
X_tokenized_test = tokenizer.texts_to_sequences(x_test)

max_len = max([len(x) for x in X_tokenized])
X_padded = pad_sequences(X_tokenized, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_tokenized_test, maxlen=max_len, padding='post')


In [56]:
y_train = np.array(y_train)
y_train_encoded = y_train + 1
y_train_encoded = to_categorical(y_train, num_classes=3)

# Predict the sentiments

In [57]:
def prediction_test(model):
  predictions = model.predict(X_test_padded)
  predicted_classes = predictions.argmax(axis=-1)
  predicted_classes = np.vectorize({2: -1, 1: 1, 0: 0}.get)(predicted_classes)  # To shift (0, 1, 2) to (-1, 0, 1)
  return predicted_classes

# Transformer Model

In [58]:
class TransformerModel(tf.keras.Model):
    def __init__(self, vocab_size, d_model, num_heads, ff_dim, num_transformer_blocks, output_dim, rate=0.1):
        # Call the constructor of the parent class (tf.keras.Model)
        super().__init__()  # This line is crucial for proper initialization

        self.embedding = layers.Embedding(vocab_size, d_model)
        self.attention = tfa.layers.MultiHeadAttention(head_size=num_heads, num_heads=num_heads)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(d_model),
        ])

        self.rnn_layer = layers.GRU(16, return_sequences=True)
        self.layer_norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.pooling = layers.GlobalAveragePooling1D()
        self.dense = layers.Dense(output_dim, activation='softmax')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reshape(x, [-1, tf.shape(x)[1], self.embedding.output_dim])

        attention_output = self.attention([x, x, x])
        x = x + attention_output
        x = self.rnn_layer(x)
        x = self.layer_norm1(x)
        x = self.ffn(x)
        x = self.dropout1(x)
        x = x + attention_output
        x = self.layer_norm2(x)
        x = self.pooling(x)
        return self.dense(x)

In [59]:
vocab_size = len(tokenizer.word_index) + 1
output_dim = 3
num_transformer_blocks = 2
d_model = 96
num_heads = 8
ff_dim = 128
dropout_rate = 0.1
learning_rate = 0.00022491452958964034

model = TransformerModel(vocab_size, d_model, num_heads, ff_dim, num_transformer_blocks, output_dim, rate=dropout_rate)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model.fit(
    X_padded, y_train_encoded,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping]
)



Epoch 1/10


201/201 [==============================] - 101s 487ms/step - loss: 0.7221 - accuracy: 0.6914 - val_loss: 0.5931 - val_accuracy: 0.7672
Epoch 2/10
201/201 [==============================] - 97s 480ms/step - loss: 0.4502 - accuracy: 0.8469 - val_loss: 0.4641 - val_accuracy: 0.8393
Epoch 3/10
201/201 [==============================] - 97s 482ms/step - loss: 0.3231 - accuracy: 0.8959 - val_loss: 0.5070 - val_accuracy: 0.8276
Epoch 4/10
201/201 [==============================] - 97s 481ms/step - loss: 0.2309 - accuracy: 0.9262 - val_loss: 0.5409 - val_accuracy: 0.8238
Epoch 5/10
201/201 [==============================] - 97s 480ms/step - loss: 0.1626 - accuracy: 0.9518 - val_loss: 0.7294 - val_accuracy: 0.8105
Epoch 6/10
201/201 [==============================] - 102s 508ms/step - loss: 0.1322 - accuracy: 0.9610 - val_loss: 0.8064 - val_accuracy: 0.7737
Epoch 7/10
201/201 [==============================] - 103s 515ms/step - loss: 0.1070 - accuracy: 0.9653 - val_loss: 0.7906 -

In [63]:
# Save only the weights
model.save_weights('transformer_weights.h5')
